<a href="https://colab.research.google.com/github/choki0715/lecture/blob/master/rag_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 임베딩 메소드 설정
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.embeddings import FastEmbedEmbeddings
# from langchain_community.embeddings import OllamaEmbeddings
# from langchain_community.embeddings import GPT4AllEmbeddings
# from gpt4all import Embed4All
# from langchain.embeddings import OpenAIEmbeddings
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain_mistralai import MistralAIEmbeddings
# from langchain_google_vertexai import VertexAIEmbeddings

# PDF 리더
# from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
# from langchain.vectorstores import FAISS

# 벡터 데이터 베이스 결정
from langchain_community.vectorstores import FAISS

# 언어모델 결정
from langchain_community.llms import Ollama
# from langchain.llms import OpenAI
# from langchain_community.llms import OpenAI
# from langchain_community.chat_models import ChatOpenAI
# from langchain.chat_models import ChatOllama

# 대화 프롬프트 메모리 관리
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# 스트림릿 정의
import streamlit as st

import tempfile

os.environ['OPENAI_API_KEY']= ""

###########################################################################
# streamlit UI
# webpage 규격 및 디자인
st.set_page_config(layout="wide")
st.markdown("<h1 style='text-align: center; color: gray;'>인구감소 대응 정책을 위한 RAG System</h1>", unsafe_allow_html=True)
st.markdown("<h2 style='text-align: center; color: gray;'> Chat with PDFs that you upload </h2>", unsafe_allow_html=True)
st.markdown("<h5 style='text-align: center; color: gray;'>오픈소스 ollama LLM 기반</h5>", unsafe_allow_html=True)
st.markdown("<h3 style='text-align: center; color: gray;'> AIDENTIFY Inc.</h3>", unsafe_allow_html=True)

# streamlit UI
###########################################################################

# # vector data 보기
# def show_vstore(vstore):
#   vector_df = store_to_df(vstore)
#   display(vector_df)

# #vector database 구축
# def store_to_df(vstore):
#   v_dict = vstore.docstore._dict
#   data_rows = []
#   for k in v_dict.keys():
#     doc_name = v_dict[k].metadata['source'].split('/')[-1]
#     page_number = doc_name.split('_')[-1]
#     content = v_dict[k].page_content
#     data_rows.append({"chunk_id":k, "document":doc_name, "page":page_number, "content":content})
#   vector_df = pd.DataFrame(data_rows)
#   return vector_df

def document_data(query, chat_history, vectorstore):

   # Loading the saved embeddings
   # new_vector_store =FAISS.load_local("vectors", embeddings)

    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

   #  llm 선정
    llm = Ollama(model="llama3")
   #  llm = Ollama(model="solar")
   #  llm = Ollama(model="qwen2")

    # langchain의 핵심
    qna = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory)

    return qna({"question":query, "chat_history":chat_history})


if __name__ == '__main__':

   #  임베딩 메소드 결정
   #  embeddings = Embed4All()
    embeddings = OpenAIEmbeddings()
   #  embeddings = OpenAIEmbeddings()
   #  embeddings = HuggingFaceEmbeddings()

    # Load Data for Retrieved Prompt
    uploaded_files = st.file_uploader("Choose a PDF file", type="PDF", accept_multiple_files=True)

    # chunk 스플릿터 정의
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap= 100, separators=["\n\n","\n"," ",""])

    temp_dir = tempfile.TemporaryDirectory()
    docs = []

    for i, uploaded_file in enumerate(uploaded_files):
      #bytes_data = uploaded_file.read()
      st.write("filename:", i, uploaded_file.name)
      temp_filepath = os.path.join(temp_dir.name, uploaded_file.name)
      with open(temp_filepath, "wb") as f:
            f.write(uploaded_file.getvalue())
      #st.write(bytes_data)
      loader = PyPDFLoader(temp_filepath)
      docs.extend(loader.load())
      # doc = loader.load()

      split_text = text_splitter.split_documents(documents = docs)
      vectorstore = FAISS.from_documents(split_text, embedding=embeddings)

    # 벡터데이터베이스 구축 확인
    print("Embeddings successfully saved in vector Database and saved locally")

    # ChatInput
    st.header(' ')
    st.header(' ')
    st.header(':blue[질문을 입력해 주세요]', divider='rainbow')
    #st.header('_Streamlit_ is :blue[cool] :sunglasses:')
    prompt = st.chat_input("Enter your questions here")

    if "user_prompt_history" not in st.session_state:
       st.session_state["user_prompt_history"]=[]
    if "chat_answers_history" not in st.session_state:
       st.session_state["chat_answers_history"]=[]
    if "chat_history" not in st.session_state:
       st.session_state["chat_history"]=[]

    if prompt:
       with st.spinner("Generating......"):
           output=document_data(query=prompt, chat_history = st.session_state["chat_history"], vectorstore=vectorstore)

          # Storing the questions, answers and chat history

           st.session_state["chat_answers_history"].append(output['answer'])
           st.session_state["user_prompt_history"].append(prompt)
           st.session_state["chat_history"].append((prompt,output['answer']))

    # Displaying the chat history

    if st.session_state["chat_answers_history"]:
       for i, j in zip(st.session_state["chat_answers_history"],st.session_state["user_prompt_history"]):
          message1 = st.chat_message("user")
          message1.write(j)
          message2 = st.chat_message("assistant")
          message2.write(i)

